In [0]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("Local File Loading").getOrCreate()


loan_df = spark.read.csv("/FileStore/tables/loan.csv", header=True, inferSchema=True)

loan_df.show(5)


+-----------+---+------+------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|  Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|    IB14001| 30|  MALE|BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|      HOUSING| 10,00,000 |      5|      42,898|               6|                 9|
|    IB14008| 44|  MALE|   PROFESSOR|       MARRIED|          6| 51000|      19999|            4|     SHOPPING|     50,000|      3|      33,999|               1|                 5|
|    IB14012| 30|FEMALE|     DENTIST|        SINGLE|          3| 58450|      27675|            

In [0]:
# Filter: Customers with income greater than 50,000 and single marital status
filtered_df = loan_df.filter((loan_df["Income"] > 50000) & (loan_df["Marital Status"] == "SINGLE"))
filtered_df.show(5)


+-----------+---+------+-----------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|       Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+-----------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|    IB14012| 30|FEMALE|          DENTIST|        SINGLE|          3| 58450|      27675|            5|   TRAVELLING|     75,000|      6|      20,876|               3|                 1|
|    IB14032| 24|  MALE|     DATA ANALYST|        SINGLE|          4| 60111|      28999|            6|   AUTOMOBILE|     35,232|      5|      33,333|               1|                 2|
|    IB14042| 25|FEMALE|           DOCTOR|        SINGLE|          4| 

In [0]:
# Example second dataset
demographics = [(1, "Urban"), (2, "Rural"), (3, "Urban")]
columns = ["Customer_ID", "Area"]

# Convert to PySpark DataFrame
demographics_df = spark.createDataFrame(demographics, columns)


# Full outer join: All rows from both DataFrames, with nulls for non-matches
outer_joined = loan_df.join(demographics_df, "Customer_ID", "outer")
outer_joined.show(5)


+-----------+---+------+------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+----+
|Customer_ID|Age|Gender|  Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|Area|
+-----------+---+------+------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+----+
|    IB14001| 30|  MALE|BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|      HOUSING| 10,00,000 |      5|      42,898|               6|                 9|null|
|    IB14008| 44|  MALE|   PROFESSOR|       MARRIED|          6| 51000|      19999|            4|     SHOPPING|     50,000|      3|      33,999|               1|                 5|null|
|    IB14012| 30|FEMALE|     DENTIST|        SINGLE|          3| 58450

In [0]:
# Total loan amount
loan_df.filter(loan_df["Expenditure"] > 50000).count()

Out[15]: 6

In [0]:
grouped_df = loan_df.groupBy("Loan Category").avg("Income")
grouped_df.show(5)


+-------------+------------------+
|Loan Category|       avg(Income)|
+-------------+------------------+
|      HOUSING| 74728.19354838709|
|   TRAVELLING| 57016.58490566038|
|  BOOK STORES|50903.142857142855|
|  AGRICULTURE|60372.666666666664|
|    GOLD LOAN| 70838.31506849315|
+-------------+------------------+
only showing top 5 rows



In [0]:
loan_df.createOrReplaceTempView("loan_data")
filtered_sql = spark.sql("""
    SELECT * 
    FROM loan_data 
    WHERE Income > 50000 AND `Marital Status` = 'SINGLE'
""")
filtered_sql.show(5)


+-----------+---+------+-----------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|       Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+-----------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|    IB14012| 30|FEMALE|          DENTIST|        SINGLE|          3| 58450|      27675|            5|   TRAVELLING|     75,000|      6|      20,876|               3|                 1|
|    IB14032| 24|  MALE|     DATA ANALYST|        SINGLE|          4| 60111|      28999|            6|   AUTOMOBILE|     35,232|      5|      33,333|               1|                 2|
|    IB14042| 25|FEMALE|           DOCTOR|        SINGLE|          4| 

In [0]:
demographics_df.createOrReplaceTempView("demographics")
outer_sql = spark.sql("""
    SELECT l.*, d.Area
    FROM loan_data l
    FULL OUTER JOIN demographics d
    ON l.Customer_ID = d.Customer_ID
""")
outer_sql.show(5)

+-----------+---+------+------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+----+
|Customer_ID|Age|Gender|  Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|Area|
+-----------+---+------+------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+----+
|    IB14001| 30|  MALE|BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|      HOUSING| 10,00,000 |      5|      42,898|               6|                 9|null|
|    IB14008| 44|  MALE|   PROFESSOR|       MARRIED|          6| 51000|      19999|            4|     SHOPPING|     50,000|      3|      33,999|               1|                 5|null|
|    IB14012| 30|FEMALE|     DENTIST|        SINGLE|          3| 58450

In [0]:
aggregation_sql = spark.sql("""
    SELECT SUM(`Expenditure`) AS Total_Loan_Amount
    FROM loan_data
""")
aggregation_sql.show()


+-----------------+
|Total_Loan_Amount|
+-----------------+
|         13243460|
+-----------------+



In [0]:
# grouped_sql = spark.sql("""
#     SELECT `Loan Category`, AVG(`Income`) AS Avg_Income
#     FROM loan_data
#     GROUP BY `Loan Category`
# """)
# grouped_sql.show()
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("Local File Loading").getOrCreate()


loan_df = spark.read.csv("/FileStore/tables/loan.csv", header=True, inferSchema=True)

loan_df.createOrReplaceTempView("loan_data")
average_loan_by_gender_sql = spark.sql("""
    SELECT Gender, AVG(`Income`) AS Avg_Income
    FROM loan_data
    GROUP BY Gender
    ORDER BY Avg_Income DESC
""")
average_loan_by_gender_sql.show()


+------+-----------------+
|Gender|       Avg_Income|
+------+-----------------+
|FEMALE|76242.93779904307|
|  MALE|61961.80694980695|
+------+-----------------+

